# Create MoPo RDF from XLSX File

## Daten verstehen

### Zusammenhang zwischen Excel und PDF Bericht

Die entscheidenden Spalten sind:

- Abschreibung (Ab)
- automatische Lokalisierung (AL)
- manuelle Lokalisierung (ML)
- Begründungstext (Bg)

- Kapitel 1: ML Kapitel 1 (Anhang 2) und Datum leer oder nach 2022 (Begründungstext muss dann immer etwas haben, 16.3317 wohl fehlerhaft)
- Kapitel 2: AL Kapitel 2 (Anhang 2) und ML leer oder ML Kapitel 2 (Anhang 2) (Begründungstext muss da sein, Abschreibedatum kann schon im Berichtsjahr sein 19.3531 Abschreibung beantragt mit Geschäft...) -- Information
- Anhang 1: AL Anhang 1 und ML leer oder ML Anhang 1 (haben alle einen Wert für "Geschäft in Erfüllung des parlamentarischen Vorstosses") oder 
- Anhang 2: Alle aus Kapitel 1 und 2?


### Typen

| Typ                         | Ab    | AL                   | ML                   | Bg   | Bsp     |
| --------------------------- | ----- | -------------------- | -------------------- | ---- | ------- |
| Hängige Geschäfte           | leer  | Anhang 2             | leer                 | leer | 22.4263 |
| Information                 | leer  | Kapitel 2 (Anhang 2) | leer                 | TXT  | 20.4341 |
| Information                 | leer  | egal                 | Kapitel 2 (Anhang 2) | TXT  | 20.4035 |
| Antrag auf Abschreibung     | leer  | egal                 | Kapitel 1 (Anhang 2) | TXT  | 20.3268 |
| Abgeschrieben mit Botschaft | Datum | Anhang 1             | leer                 | TXT  | 21.3664 |
| Abgeschrieben mit Botschaft | Datum | egal                 | Anhang 1             | TXT  | 16.3884 |
| Abgeschrieben mit Botschaft | Datum | Anhang 1             | leer                 | leer | 20.3917 |

Komische Fälle

| Typ                          | Ab   | AL       | ML   | Bg  | Bsp     |
| ---------------------------- | ---- | -------- | ---- | --- | ------- |
| Abgelehnte Abschreibungen??? | leer | Anhang 2 | leer | TXT | 20.3485 |


- Anträge auf Abschreibung: Noch kein Datum in `Abschreibung`, aber Begründungstext
  - Bsp: 20.4341

- Regulär abgeschrieben: Datum in `Abschreibung` und Begründungstext
  - Bsp: 22.3385
- Nicht regulär abgeschrieben: Datum in `Abschreibung` aber kein Begründungstext
  - Bsp: 22.4022

### Datumsspalten

- `Einreichung` haben alle Zeilen
- `Überweisung` haben alle Zeilen
- `Abschreibung` haben nicht alle Zeilen, falls leer kann Begründung vorhanden sein (Antrag auf Abschreibung) oder noch leer (noch hängig)

### Spalten "Lokalisierung im Bericht"

- Werte in der "manuellen" Spalte, übersteuren die "automatischen"
- falls kein Wert in "manuell", dann gilt "automatisch"

### Mögliche Fehler in den Daten

- 20.3128 (im Jahr 2021) hat in allen Sprachen kein Überweisungsdatum, gemäss: https://www.parlament.ch/de/ratsbetrieb/suche-curia-vista/geschaeft?AffairId=20203128 wohl 04.05.2020 --> so eingefügt
- 19.3734 (im Jahr 2022) hat in der manuellen Lokalisierung: Kapitel 1 (Anhang 2), müsste aber wohl leer sein
- 16.3317 (im Jahr 2022) hat keine Begründung

### Fragen

Wie kann eine abgelehnte MoPo Abschreibung erkannt werden?

## Merging the yearly XLSX

Idea: If an identifier is in the newest year, take all the info from there (there shouldn't be any changed information in the later years, only added), only take infos from older years if they are not present in the newer files.

## Read XLSX for 2022 and Combine Languages

In [1]:
import pandas as pd
import os

dtype_dict = {'CuriaId': str,
              'Curia ID': str}

data_de = pd.read_excel("../local/data/Export-2022-De.xlsx", dtype=dtype_dict, parse_dates=[5, 6, 7], date_format='%d.%m.%Y')
print(data_de.shape)
display(data_de.head())

(875, 18)


,CuriaId,Vorstossart,Ursprungsrat,Autor,Titel,Einreichungsdatum,Überweisungsdatum,Abschreibungsdatum,Federführendes Departement,Amt/Direktion,Lokalisierung im Bericht (automatisch),Lokalisierung im Bericht (manuell),Konnexe Vorstösse,Geschäft in Erfüllung des parlamentarischen Vorstosses,Begründungstext Deutsch,Begründungstext Französisch,Begründungstext Italienisch,Eingereichter Text
0,22.4263,Motion,Ständerat,Kommission für Verkehr und Fernmeldewesen SR,"Rasche Gewährleistung einer ausgewogenen, leis...",2022-10-27,2022-12-12,NaT,UVEK,BAV,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,Gemäss den Beschlüssen der Bundesversammlung i...
1,22.4262,Postulat,Nationalrat,Kommission für soziale Sicherheit und Gesundhe...,Ambulant vor stationär für Menschen mit Behind...,2022-10-21,2022-12-15,NaT,EDI,BSV,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt zu prüfen, inwie..."
2,22.4257,Motion,Nationalrat,Kommission für Verkehr und Fernmeldewesen NR,"Rasche Gewährleistung einer ausgewogenen, leis...",2022-10-17,2022-12-12,NaT,UVEK,BAV,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,Gemäss den Beschlüssen der Bundesversammlung i...
3,22.4252,Postulat,Ständerat,Kommission für Wirtschaft und Abgaben SR,Wettbewerbssituation im Lebensmittelmarkt,2022-10-10,2022-12-13,NaT,WBF,BLW,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,Vor dem Hintergrund der starken Konzentration ...
4,22.4250,Motion,Ständerat,Geschäftsprüfungskommission SR,Erhöhung der Obergrenze der Gerichtsgebühren d...,2022-08-30,2022-12-14,NaT,EJPD,BJ,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt, die gesetzliche..."


In [2]:
data_fr = pd.read_excel("../local/data/Export-2022-Fr.xlsx", dtype=dtype_dict, parse_dates=[5, 6, 7], date_format='%d.%m.%Y')
print(data_fr.shape)
display(data_fr.head())

(875, 18)


,Curia ID,Type d’intervention,Premier conseil,Auteur,Titre,Date de dépôt,Date de transmission,Date de classement,Département responsable,Office/direction,Localisation dans le rapport (automatique),Localisation dans le rapport (manuel),Interventions apparentées,Objet en exécution de l’intervention parlementaire,Développement en allemand,Développement en français,Développement en italien,Texte déposé
0,22.4263,Motion,Conseil des États,Commission des transports et des télécommunica...,Garantir rapidement un axe ferroviaire est-oue...,2022-10-27,2022-12-12,NaT,DETEC,OFT,Annexe 2,NaN,NaN,NaN,NaN,NaN,NaN,Conformément aux décisions prises par l'Assemb...
1,22.4262,Postulat,Conseil national,Commission de la sécurité sociale et de la san...,L'ambulatoire avant le stationnaire pour les p...,2022-10-21,2022-12-15,NaT,DFI,OFAS,Annexe 2,NaN,NaN,NaN,NaN,NaN,NaN,Le Conseil fédéral est chargé d'examiner dans ...
2,22.4257,Motion,Conseil national,Commission des transports et des télécommunica...,Garantir rapidement un axe ferroviaire est-oue...,2022-10-17,2022-12-12,NaT,DETEC,OFT,Annexe 2,NaN,NaN,NaN,NaN,NaN,NaN,Conformément aux décisions prises par l'Assemb...
3,22.4252,Postulat,Conseil des États,Commission de l'économie et des redevances CE,Concurrence sur le marché de l'alimentation,2022-10-10,2022-12-13,NaT,DEFR,OFAG,Annexe 2,NaN,NaN,NaN,NaN,NaN,NaN,"Étant donné la forte concentration, en Suisse,..."
4,22.4250,Motion,Conseil des États,Commission de gestion CE,Relèvement des plafonds des émoluments judicia...,2022-08-30,2022-12-14,NaT,DFJP,OFJ,Annexe 2,NaN,NaN,NaN,NaN,NaN,NaN,Le Conseil fédéral est chargé d'adapter les ba...


In [3]:
data = pd.merge(data_de, data_fr[['Curia ID','Titre', 'Texte déposé']], left_on='CuriaId', right_on='Curia ID', how='inner')
data.drop(columns=['Curia ID'], inplace=True)
print(data.shape)
display(data.head())

(875, 20)


,CuriaId,Vorstossart,Ursprungsrat,Autor,Titel,Einreichungsdatum,Überweisungsdatum,Abschreibungsdatum,Federführendes Departement,Amt/Direktion,Lokalisierung im Bericht (automatisch),Lokalisierung im Bericht (manuell),Konnexe Vorstösse,Geschäft in Erfüllung des parlamentarischen Vorstosses,Begründungstext Deutsch,Begründungstext Französisch,Begründungstext Italienisch,Eingereichter Text,Titre,Texte déposé
0,22.4263,Motion,Ständerat,Kommission für Verkehr und Fernmeldewesen SR,"Rasche Gewährleistung einer ausgewogenen, leis...",2022-10-27,2022-12-12,NaT,UVEK,BAV,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,Gemäss den Beschlüssen der Bundesversammlung i...,Garantir rapidement un axe ferroviaire est-oue...,Conformément aux décisions prises par l'Assemb...
1,22.4262,Postulat,Nationalrat,Kommission für soziale Sicherheit und Gesundhe...,Ambulant vor stationär für Menschen mit Behind...,2022-10-21,2022-12-15,NaT,EDI,BSV,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt zu prüfen, inwie...",L'ambulatoire avant le stationnaire pour les p...,Le Conseil fédéral est chargé d'examiner dans ...
2,22.4257,Motion,Nationalrat,Kommission für Verkehr und Fernmeldewesen NR,"Rasche Gewährleistung einer ausgewogenen, leis...",2022-10-17,2022-12-12,NaT,UVEK,BAV,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,Gemäss den Beschlüssen der Bundesversammlung i...,Garantir rapidement un axe ferroviaire est-oue...,Conformément aux décisions prises par l'Assemb...
3,22.4252,Postulat,Ständerat,Kommission für Wirtschaft und Abgaben SR,Wettbewerbssituation im Lebensmittelmarkt,2022-10-10,2022-12-13,NaT,WBF,BLW,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,Vor dem Hintergrund der starken Konzentration ...,Concurrence sur le marché de l'alimentation,"Étant donné la forte concentration, en Suisse,..."
4,22.4250,Motion,Ständerat,Geschäftsprüfungskommission SR,Erhöhung der Obergrenze der Gerichtsgebühren d...,2022-08-30,2022-12-14,NaT,EJPD,BJ,Anhang 2,NaN,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt, die gesetzliche...",Relèvement des plafonds des émoluments judicia...,Le Conseil fédéral est chargé d'adapter les ba...


In [4]:
data_it = pd.read_excel("../local/data/Export-2022-It.xlsx", dtype=dtype_dict, parse_dates=[5, 6, 7], date_format='%d.%m.%Y')
print(data_it.shape)
display(data_it.head())

(875, 18)


,Curia ID,Tipo di intervento,Camera iniziale,Autore,Titolo,Data del deposito,Data della trasmissione,Data dello stralcio,Dipartimento competente,Ufficio/direzione,Posizione nel rapporto (automatico),Posizione nel rapporto (manuale),Interventi connessi,Oggetto in adempimento dell'intervento parlamentare,Motivazione in tedesco,Motivazione in francese,Motivazione in italiano,Testo depositato
0,22.4263,Mozione,Consiglio degli Stati,Commissione dei trasporti e delle telecomunica...,Garantire in tempi rapidi un asse ferroviario ...,2022-10-27,2022-12-12,NaT,DATEC,UFT,Allegato 2,NaN,NaN,NaN,NaN,NaN,NaN,Conformemente alle decisioni prese dall'Assemb...
1,22.4262,Postulato,Consiglio nazionale,Commissione della sicurezza sociale e della sa...,Preferire le cure ambulatoriali alle stazionar...,2022-10-21,2022-12-15,NaT,DFI,UFAS,Allegato 2,NaN,NaN,NaN,NaN,NaN,NaN,Il Consiglio federale è incaricato di esaminar...
2,22.4257,Mozione,Consiglio nazionale,Commissione dei trasporti e delle telecomunica...,Garantire in tempi rapidi un asse ferroviario ...,2022-10-17,2022-12-12,NaT,DATEC,UFT,Allegato 2,NaN,NaN,NaN,NaN,NaN,NaN,Conformemente alle decisioni prese dall'Assemb...
3,22.4252,Postulato,Consiglio degli Stati,Commissione dell'economia e dei tributi CS,Concorrenza sul mercato delle derrate alimentari,2022-10-10,2022-12-13,NaT,DEFR,UFAG,Allegato 2,NaN,NaN,NaN,NaN,NaN,NaN,In considerazione della forte concentrazione n...
4,22.4250,Mozione,Consiglio degli Stati,Commissione della gestione CS,Aumento dei limiti massimi delle tasse di gius...,2022-08-30,2022-12-14,NaT,DFGP,UFG,Allegato 2,NaN,NaN,NaN,NaN,NaN,NaN,Il Consiglio federale è incaricato di adeguare...


In [5]:
data = pd.merge(data, data_it[['Curia ID','Titolo','Testo depositato']], left_on='CuriaId', right_on='Curia ID', how='inner')
data.drop(columns=['Curia ID'], inplace=True)
print(data.shape)
display(data.head())

(875, 22)


,CuriaId,Vorstossart,Ursprungsrat,Autor,Titel,Einreichungsdatum,Überweisungsdatum,Abschreibungsdatum,Federführendes Departement,Amt/Direktion,...,Konnexe Vorstösse,Geschäft in Erfüllung des parlamentarischen Vorstosses,Begründungstext Deutsch,Begründungstext Französisch,Begründungstext Italienisch,Eingereichter Text,Titre,Texte déposé,Titolo,Testo depositato
0,22.4263,Motion,Ständerat,Kommission für Verkehr und Fernmeldewesen SR,"Rasche Gewährleistung einer ausgewogenen, leis...",2022-10-27,2022-12-12,NaT,UVEK,BAV,...,NaN,NaN,NaN,NaN,NaN,Gemäss den Beschlüssen der Bundesversammlung i...,Garantir rapidement un axe ferroviaire est-oue...,Conformément aux décisions prises par l'Assemb...,Garantire in tempi rapidi un asse ferroviario ...,Conformemente alle decisioni prese dall'Assemb...
1,22.4262,Postulat,Nationalrat,Kommission für soziale Sicherheit und Gesundhe...,Ambulant vor stationär für Menschen mit Behind...,2022-10-21,2022-12-15,NaT,EDI,BSV,...,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt zu prüfen, inwie...",L'ambulatoire avant le stationnaire pour les p...,Le Conseil fédéral est chargé d'examiner dans ...,Preferire le cure ambulatoriali alle stazionar...,Il Consiglio federale è incaricato di esaminar...
2,22.4257,Motion,Nationalrat,Kommission für Verkehr und Fernmeldewesen NR,"Rasche Gewährleistung einer ausgewogenen, leis...",2022-10-17,2022-12-12,NaT,UVEK,BAV,...,NaN,NaN,NaN,NaN,NaN,Gemäss den Beschlüssen der Bundesversammlung i...,Garantir rapidement un axe ferroviaire est-oue...,Conformément aux décisions prises par l'Assemb...,Garantire in tempi rapidi un asse ferroviario ...,Conformemente alle decisioni prese dall'Assemb...
3,22.4252,Postulat,Ständerat,Kommission für Wirtschaft und Abgaben SR,Wettbewerbssituation im Lebensmittelmarkt,2022-10-10,2022-12-13,NaT,WBF,BLW,...,NaN,NaN,NaN,NaN,NaN,Vor dem Hintergrund der starken Konzentration ...,Concurrence sur le marché de l'alimentation,"Étant donné la forte concentration, en Suisse,...",Concorrenza sul mercato delle derrate alimentari,In considerazione della forte concentrazione n...
4,22.4250,Motion,Ständerat,Geschäftsprüfungskommission SR,Erhöhung der Obergrenze der Gerichtsgebühren d...,2022-08-30,2022-12-14,NaT,EJPD,BJ,...,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt, die gesetzliche...",Relèvement des plafonds des émoluments judicia...,Le Conseil fédéral est chargé d'adapter les ba...,Aumento dei limiti massimi delle tasse di gius...,Il Consiglio federale è incaricato di adeguare...


## Read XLSX for 2021 and Combine Languages and merge with 2022

In [55]:
dtype_dict = {'CuriaId': str,
              'Curia ID': str}

data_de = pd.read_excel("../local/data/Export-2021-De.xlsx", dtype=dtype_dict, parse_dates=[5, 6, 7], date_format='%d.%m.%Y')
print(data_de.shape)

data_fr = pd.read_excel("../local/data/Export-2021-Fr.xlsx", dtype=dtype_dict, parse_dates=[5, 6, 7], date_format='%d.%m.%Y')
print(data_fr.shape)

data_21 = pd.merge(data_de, data_fr[['Curia ID','Titre', 'Texte déposé']], left_on='CuriaId', right_on='Curia ID', how='inner')
data_21.drop(columns=['Curia ID'], inplace=True)
print(data_21.shape)

data_it = pd.read_excel("../local/data/Export-2021-It.xlsx", dtype=dtype_dict, parse_dates=[5, 6, 7], date_format='%d.%m.%Y')
print(data_it.shape)

data_21 = pd.merge(data_21, data_it[['Curia ID','Titolo','Testo depositato']], left_on='CuriaId', right_on='Curia ID', how='inner')
data_21.drop(columns=['Curia ID'], inplace=True)
print(data_21.shape)
display(data_21.head())

(819, 18)
(819, 18)
(819, 20)
(819, 18)
(819, 22)


,CuriaId,Vorstossart,Ursprungsrat,Autor,Titel,Einreichungsdatum,Überweisungsdatum,Abschreibungsdatum,Federführendes Departement,Amt/Direktion,...,Konnexe Vorstösse,Geschäft in Erfüllung des parlamentarischen Vorstosses,Begründungstext Deutsch,Begründungstext Französisch,Begründungstext Italienisch,Eingereichter Text,Titre,Texte déposé,Titolo,Testo depositato
0,21.4345,Postulat,Ständerat,"Kommission für Wissenschaft, Bildung und Kultu...",Züchtungsverfahren mit Genom-Editierungsmethoden,2021-11-16,2021-12-02,NaT,UVEK,BAFU,...,NaN,NaN,NaN,NaN,NaN,Der Bundesrat erstattet dem Parlament innert J...,Procédés de sélection par édition génomique,"Le Conseil fédéral présente au Parlement, dans...",Procedure di selezione con metodi di editing g...,"Il Consiglio federale presenta al Parlamento, ..."
1,21.4219,Postulat,Nationalrat,Marco Romano,Bekämpfung der internationalen organisierten K...,2021-09-30,2021-12-17,NaT,EJPD,fedpol,...,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt, einen Bericht z...",Lutte contre la criminalité internationale org...,Le Conseil fédéral est chargé de présenter un ...,Lotta alla criminalità organizzata internazion...,Il Consiglio federale è incaricato di esaminar...
2,21.4176,Postulat,Nationalrat,Judith Bellaiche,Cyberrisiken im All,2021-09-30,2021-12-17,NaT,VBS,GS-VBS,...,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird gebeten, eine Auslegeordnun...",Cyberrisques dans l'espace,Le Conseil fédéral est prié d'établir une vue ...,Ciber-rischi nello spazio extra-atmosferico,Il Consiglio federale è invitato ad analizzare...
3,21.4141,Postulat,Nationalrat,Andri Silberschmidt,Evaluation der Gerichtspraxis nach der Revisio...,2021-09-29,2021-12-17,NaT,EJPD,BJ,...,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt, eine Evaluation...",Évaluation de la pratique des tribunaux suite ...,Le Conseil fédéral est chargé d'évaluer la pra...,Valutazione della prassi giudiziaria dopo la r...,Il Consiglio federale è incaricato di valutare...
4,21.4079,Postulat,Nationalrat,Philipp Kutter,Wirkungsüberprüfung der Steuerreform STAF,2021-09-23,2021-12-17,NaT,EFD,ESTV,...,NaN,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt, die Umsetzung d...",Analyse des effets de la réforme fiscale RFFA,Le Conseil fédéral est chargé de procéder à un...,Verificare l’efficacia della riforma fiscale RFFA,Il Consiglio federale è incaricato di valutare...


### Concatenate Dataframes

In [56]:
# Step 1: Extract the identifier column from both DataFrames
ids_22 = data.CuriaId

# Step 2: Filter rows in df2 where the 'id' is not in df1
data_21_filtered = data_21[~data_21['CuriaId'].isin(ids_22)]

# Step 3: Concatenate df1 with the filtered df2
data = pd.concat([data, data_21_filtered], ignore_index=True)

# Display the result
print(data.shape)

(1042, 22)


## Transform Data to RDF

### get department for abbrevation

In [6]:
import requests

def org_lookup_table(level):
    
    if level == "department":

        sparql_query = """

        PREFIX vl: <https://version.link/>
        PREFIX schema: <http://schema.org/>
        SELECT DISTINCT * WHERE {
        
        ?org schema:parentOrganization <https://ld.admin.ch/FC>;
            a vl:Identity;
            schema:alternateName ?abbr.
        
        FILTER(lang(?abbr) = "de")
        }

        """
    elif level == "office":
        
        sparql_query = """

        PREFIX vl: <https://version.link/>
        PREFIX schema: <http://schema.org/>
        SELECT DISTINCT * WHERE {
        
        ?org schema:parentOrganization/schema:parentOrganization <https://ld.admin.ch/FC>;
            a vl:Identity;
            schema:alternateName ?abbr.
        
        FILTER(lang(?abbr) = "de")
        }

        """

    encoded_query = {"query": sparql_query}

    headers = {
        "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
        "Accept": "application/sparql-results+json" 
    }

    response = requests.post("https://ld.admin.ch/query", 
                             data=encoded_query, 
                             headers=headers)
    
    if response.status_code == 200:

        response.encoding = "utf-8"

        data = response.json()

        data = data["results"]["bindings"]

        data = [{'org': d['org']['value'], 'abbr': d['abbr']['value']} for d in data]

        data = pd.DataFrame(data)

        data.set_index('abbr', inplace=True)

    return data

dep_lookup = org_lookup_table(level = "department")
office_lookup = org_lookup_table(level = "office")
display(dep_lookup)
display(office_lookup)

,org
abbr,
EDA,https://ld.admin.ch/department/I
EDI,https://ld.admin.ch/department/II
EJPD,https://ld.admin.ch/department/III
VBS,https://ld.admin.ch/department/IV
EFD,https://ld.admin.ch/department/V
WBF,https://ld.admin.ch/department/VI
UVEK,https://ld.admin.ch/department/VII
BK,https://ld.admin.ch/FCh


,org
abbr,
GS-EDA,https://ld.admin.ch/office/I.1.1
DV,https://ld.admin.ch/office/I.1.4
DEZA,https://ld.admin.ch/office/I.1.5
DR,https://ld.admin.ch/office/I.1.7
KD,https://ld.admin.ch/office/I.1.8
...,...
KS,https://ld.admin.ch/ou/10002695
R,https://ld.admin.ch/ou/10010826
BK,https://ld.admin.ch/ou/10010833


In [12]:
from rdflib import Graph, URIRef, Literal, BNode, Namespace
import re

g = Graph(bind_namespaces="none") # no predefined namespaces (e.g. RDF, RDFS, OWL)

prov = Namespace("http://www.w3.org/ns/prov#")
g.bind("prov", prov)

rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
g.bind("rdf", rdf)

schema = Namespace("http://schema.org/")
g.bind("schema", schema)

chpaf = Namespace("https://ch.paf.link/")
g.bind("chpaf", chpaf)

paf = Namespace("https://paf.link/")
g.bind("paf", paf)

dcterm = Namespace("http://purl.org/dc/terms/")
g.bind("dcterm", dcterm)

# define motion and postulate

motion = URIRef("https://ch.paf.link/Motion")

g.add((motion, rdf.type, URIRef("https://paf.link/Affair")))
g.add((motion, schema.name, Literal("Motion", lang='de')))
g.add((motion, schema.name, Literal("motion", lang='fr')))
g.add((motion, schema.name, Literal("mozione", lang='it')))
g.add((motion, schema.name, Literal("moziun", lang='rm')))
g.add((motion, schema.name, Literal("motion", lang='en')))
g.add((motion, chpaf.termdat, URIRef("https://register.ld.admin.ch/termdat/109123")))
g.add((motion, chpaf.parlApiId, Literal("5", datatype="http://www.w3.org/2001/XMLSchema#integer")))

postulate = URIRef("https://ch.paf.link/Postulate")

g.add((postulate, rdf.type, URIRef("https://paf.link/Affair")))
g.add((postulate, schema.name, Literal("Postulat", lang='de')))
g.add((postulate, schema.name, Literal("postulat", lang='fr')))
g.add((postulate, schema.name, Literal("postulato", lang='it')))
g.add((postulate, schema.name, Literal("postulat", lang='rm')))
g.add((postulate, schema.name, Literal("postulate", lang='en')))
g.add((postulate, chpaf.termdat, URIRef("https://register.ld.admin.ch/termdat/109124")))
g.add((postulate, chpaf.parlApiId, Literal("6", datatype="http://www.w3.org/2001/XMLSchema#integer")))

# line by line processing

def mopo(line, g):
    
    # view_point

    view_point = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}")

    g.add((view_point, rdf.type, paf.ViewPoint))

    # registration_activity

    registration_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/registration-activity")
    
    g.add((registration_activity, rdf.type, prov.Activity))
    g.add((registration_activity, paf.activityType, chpaf.Registration))
    g.add((registration_activity, prov.startedAtTime, Literal(line['Überweisungsdatum'].strftime('%Y-%m-%d'), datatype="http://www.w3.org/2001/XMLSchema#date")))

    # identifier_entity

    identifier_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/identifier-entity")

    g.add((identifier_entity, rdf.type, prov.Entity))
    g.add((identifier_entity, prov.wasGeneratedBy, registration_activity))
    g.add((identifier_entity, rdf.predicate, schema.identifier))
    g.add((identifier_entity, schema.identifier, Literal(line['CuriaId'])))
    g.add((view_point, dcterm.hasPart, identifier_entity))

    # affair_type_entity

    affair_type_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/affair-type-entity")

    g.add((affair_type_entity, rdf.type, prov.Entity))
    g.add((affair_type_entity, prov.wasGeneratedBy, registration_activity))
    g.add((affair_type_entity, rdf.predicate, paf.affairType))
    g.add((affair_type_entity, paf.affairType, motion if line['Vorstossart'] == "Motion" else postulate))
    g.add((view_point, dcterm.hasPart, affair_type_entity))

    # source_council_entity

    source_council_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/source-council-entity")

    g.add((source_council_entity, rdf.type, prov.Entity))
    g.add((source_council_entity, prov.wasGeneratedBy, registration_activity))
    g.add((source_council_entity, rdf.predicate, chpaf.sourceCouncil))
    g.add((source_council_entity, chpaf.sourceCouncil, URIRef("https://politics.ld.admin.ch/council/N" if line['Ursprungsrat'] == "Nationalrat" else "https://politics.ld.admin.ch/council/S")))
    g.add((view_point, dcterm.hasPart, source_council_entity))

    # author_entity

    # author is modelled as entity because we start with registration acitivty

    author_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/autor-entity")

    g.add((author_entity, rdf.type, prov.Entity))
    g.add((author_entity, prov.wasGeneratedBy, registration_activity))
    g.add((author_entity, rdf.predicate, schema.author))
    g.add((author_entity, schema.author, Literal(line['Autor'])))
    g.add((view_point, dcterm.hasPart, author_entity))

    # title_entity

    title_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/title-entity")

    g.add((title_entity, rdf.type, prov.Entity))
    g.add((title_entity, prov.wasGeneratedBy, registration_activity))
    g.add((title_entity, rdf.predicate, schema.title))
    g.add((title_entity, schema.title, Literal(line['Titel'], lang='de')))
    g.add((title_entity, schema.title, Literal(line['Titre'], lang='fr')))
    g.add((title_entity, schema.title, Literal(line['Titolo'], lang='it')))
    g.add((view_point, dcterm.hasPart, title_entity))

    # description_entity

    description_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/description-entity")

    g.add((description_entity, rdf.type, prov.Entity))
    g.add((description_entity, prov.wasGeneratedBy, registration_activity))
    g.add((description_entity, rdf.predicate, schema.description))
    g.add((description_entity, schema.description, Literal(line['Eingereichter Text'], lang='de')))
    g.add((description_entity, schema.description, Literal(line['Texte déposé'], lang='fr')))
    g.add((description_entity, schema.description, Literal(line['Testo depositato'], lang='it')))
    g.add((view_point, dcterm.hasPart, description_entity))

    # managing_department_entity

    managing_department_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/managing-department-entity")

    g.add((managing_department_entity, rdf.type, prov.Entity))
    g.add((managing_department_entity, prov.wasGeneratedBy, registration_activity))
    g.add((managing_department_entity, rdf.predicate, chpaf.department))
    g.add((managing_department_entity, chpaf.department, URIRef(dep_lookup.loc[line['Federführendes Departement'], 'org']) if line['Federführendes Departement'] in dep_lookup.index else Literal(line['Federführendes Departement'])))
    g.add((view_point, dcterm.hasPart, managing_department_entity))

    # office_entity

    office_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/office-entity")

    g.add((office_entity, rdf.type, prov.Entity))
    g.add((office_entity, prov.wasGeneratedBy, registration_activity))
    g.add((office_entity, rdf.predicate, chpaf.office))
    g.add((office_entity, chpaf.office, URIRef(office_lookup.loc[line['Amt/Direktion'], 'org']) if line['Amt/Direktion'] in office_lookup.index else Literal(line['Amt/Direktion'])))
    g.add((view_point, dcterm.hasPart, office_entity))

    # chapter 1 MoPo Bericht
    
    if (line['Lokalisierung im Bericht (automatisch)'] == "Kapitel 1 (Anhang 2)" and pd.isna(line['Lokalisierung im Bericht (manuell)'])) or line['Lokalisierung im Bericht (manuell)'] == "Kapitel 1 (Anhang 2)":
        
        # proposal_creation_activity

        proposal_creation_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/proposal-creation-activity")

        g.add((proposal_creation_activity, rdf.type, prov.Activity))
        g.add((proposal_creation_activity, paf.activityType, chpaf.ProposalCreation))
        g.add((proposal_creation_activity, prov.wasInformedBy, registration_activity))
        g.add((proposal_creation_activity, prov.used, identifier_entity))
        g.add((proposal_creation_activity, prov.used, affair_type_entity))
        g.add((proposal_creation_activity, prov.used, source_council_entity))
        g.add((proposal_creation_activity, prov.used, author_entity))
        g.add((proposal_creation_activity, prov.used, title_entity))
        g.add((proposal_creation_activity, prov.used, description_entity))
        g.add((proposal_creation_activity, prov.used, managing_department_entity))
        g.add((proposal_creation_activity, prov.used, office_entity))

        # answer_entity

        answer_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/answer-entity")

        g.add((answer_entity, rdf.type, prov.Entity))
        g.add((answer_entity, prov.wasGeneratedBy, proposal_creation_activity))
        g.add((answer_entity, rdf.predicate, chpaf.mopoAnswer))
        g.add((answer_entity, chpaf.mopoAnswer, Literal(line['Begründungstext Deutsch'], lang='de')))
        g.add((answer_entity, chpaf.mopoAnswer, Literal(line['Begründungstext Französisch'], lang='fr')))
        g.add((answer_entity, chpaf.mopoAnswer, Literal(line['Begründungstext Italienisch'], lang='it')))
        g.add((view_point, dcterm.hasPart, answer_entity))

        # if there was a postulate report
        if "Postulatsbericht" in str(line["Begründungstext Deutsch"]) or "Bericht" in str(line["Begründungstext Deutsch"]):
            parts = line["CuriaId"].split(".")
            g.add((answer_entity, paf.meta, chpaf.postulateReport))
            g.add((answer_entity, chpaf.postulateReport, Literal(f"https://www.parlament.ch/centers/eparl/curia/20{parts[0]}/20{parts[0]}{parts[1]}/Bericht BR D.pdf", lang='de')))
            g.add((answer_entity, chpaf.postulateReport, Literal(f"https://www.parlament.ch/centers/eparl/curia/20{parts[0]}/20{parts[0]}{parts[1]}/Bericht BR F.pdf", lang='fr')))

        # if there is a link to the "Amtliche Sammlung" (AS YYYY xyz)
        pattern = r'AS (\d{4}) (\d+)(\D)'
        matches = re.findall(pattern, str(line['Begründungstext Deutsch']))
        for match in matches:
            g.add((answer_entity, paf.meta, chpaf.officialCollection))
            g.add((answer_entity, chpaf.officialCollection, Literal(f"https://www.fedlex.admin.ch/eli/oc/{match[0]}/{match[1]}/de", lang='de')))
            g.add((answer_entity, chpaf.officialCollection, Literal(f"https://www.fedlex.admin.ch/eli/oc/{match[0]}/{match[1]}/fr", lang='fr')))
            g.add((answer_entity, chpaf.officialCollection, Literal(f"https://www.fedlex.admin.ch/eli/oc/{match[0]}/{match[1]}/it", lang='it')))

        # proposal_entity

        proposal_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/proposal-entity")

        g.add((proposal_entity, rdf.type, prov.Entity))
        g.add((proposal_entity, prov.wasGeneratedBy, proposal_creation_activity))
        g.add((proposal_entity, rdf.predicate, chpaf.proposal))
        g.add((proposal_entity, chpaf.proposal, chpaf.Abandonment))
        g.add((view_point, dcterm.hasPart, proposal_entity))

        # proposal_activity

        proposal_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/proposal-activity")

        g.add((proposal_activity, rdf.type, prov.Activity))
        g.add((proposal_activity, paf.activityType, chpaf.Proposal))
        g.add((proposal_activity, prov.wasInformedBy, proposal_creation_activity))

        proposal_activty_qa1 = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/proposal-activity/qa1")

        g.add((proposal_activity, prov.qualifiedAssociation, proposal_activty_qa1))
        g.add((proposal_activty_qa1, rdf.type, prov.Association))
        g.add((proposal_activty_qa1, prov.agent, URIRef("https://ld.admin.ch/FC")))
        g.add((proposal_activty_qa1, prov.hadRole, paf.Submitter))

        proposal_activty_qa2 = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/proposal-activity/qa2")
        g.add((proposal_activity, prov.qualifiedAssociation, proposal_activty_qa2))
        g.add((proposal_activty_qa2, rdf.type, prov.Association))
        g.add((proposal_activty_qa2, prov.agent, URIRef("https://ld.admin.ch/FA")))
        g.add((proposal_activty_qa2, prov.hadRole, paf.Receiver))

        g.add((proposal_activity, prov.used, identifier_entity))
        g.add((proposal_activity, prov.used, affair_type_entity))
        g.add((proposal_activity, prov.used, source_council_entity))
        g.add((proposal_activity, prov.used, author_entity))
        g.add((proposal_activity, prov.used, title_entity))
        g.add((proposal_activity, prov.used, description_entity))
        g.add((proposal_activity, prov.used, managing_department_entity))
        g.add((proposal_activity, prov.used, office_entity))

        g.add((proposal_activity, prov.used, answer_entity))
        g.add((proposal_activity, prov.used, proposal_entity))

        # if MoPo was accepted

        if pd.notna(line['Abschreibungsdatum']):

            # decision_activity

            decision_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/decision-activity")

            g.add((decision_activity, rdf.type, prov.Activity))
            g.add((decision_activity, paf.activityType, chpaf.Decision))
            g.add((decision_activity, prov.startedAtTime, Literal(line['Abschreibungsdatum'].strftime('%Y-%m-%d'), datatype="http://www.w3.org/2001/XMLSchema#date")))
            g.add((decision_activity, prov.wasInformedBy, proposal_activity))
            g.add((decision_activity, prov.used, identifier_entity))
            g.add((decision_activity, prov.used, affair_type_entity))
            g.add((decision_activity, prov.used, source_council_entity))
            g.add((decision_activity, prov.used, author_entity))
            g.add((decision_activity, prov.used, title_entity))
            g.add((decision_activity, prov.used, description_entity))
            g.add((decision_activity, prov.used, managing_department_entity))
            g.add((decision_activity, prov.used, office_entity))
            g.add((decision_activity, prov.used, answer_entity))
            g.add((decision_activity, prov.used, proposal_entity))
            g.add((decision_activity, paf.proposalActivity, proposal_activity))

            # decision_entity

            decision_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/decision-entity")

            g.add((decision_entity, rdf.type, prov.Entity))
            g.add((decision_entity, prov.wasGeneratedBy, decision_activity))
            g.add((decision_entity, rdf.predicate, paf.acceptance))
            g.add((decision_entity, paf.acceptance, paf.Accepted))
            g.add((view_point, dcterm.hasPart, decision_entity))

    return g    

data.apply(mopo, args=(g,), axis=1)

def write_ttl(g, filename):
    
    with open("../examples/" + filename + ".ttl", 'w', encoding = "utf-8") as ttl_file:
        ttl_file.write(g.serialize())

write_ttl(g, "mopo")